In [ ]:
!git clone https://github.com/westlund4228/amc_ai502.git
%cd amc_ai502

Cloning into 'amc_ai502'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 2), reused 31 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 42.31 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/amc_ai502


In [ ]:
!pip install tensorboardX torch-pruning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
!python amc_fine_tune.py \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --batch_size=256 \
    --n_worker=2 \
    --n_epoch=2 \
    --lr=0.05 \
    --lr_type=cos \
    --wd=4e-5 \
    --seed=2025 \
    --data_root=./data


In [ ]:
!python scripts/download_mobilenet_weights.py

Model saved to models/state_dict/mobilenetv1_cifar.pth.tar


In [ ]:
!python amc_search.py \
    --job=train \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.99 \
    --acc_metric=acc1 \
    --lbound=0.95 \
    --rbound=1 \
    --train_episode=4 \
    --reward=acc_reward \
    --data_root=./data \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --seed=2018

=> Building model..
=> Loading checkpoint from /content/amc2/models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
DEBUG: train/val overlap size: 0
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 97.860%
=> original weight size: 0.8118 M param
=> original FLOPs: 15.1685 M
=> Saving logs to ./logs/mobilenet_cifar_cifar10_r0.99_search-run3
=> Output path: ./logs/mobilenet_cifar_cifar10_r0.99_search-run3...
** Actual replay buffer size: 900
 New best reward: 0.9588, acc: 95.8800, compress: 0.9099
 New

In [ ]:
!python amc_search.py \
    --job=export \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --data_root=./data \
    --ckpt_path=./models/state_dict/mobilenetv1_cifar.pth.tar \
    --n_calibration_batches=25 \
    --policy_path=./logs/mobilenet_cifar_cifar10_r0.99_search-run3/best_policy.txt \
    --export_path=./checkpoints/mobilenet_export.pt \
    --seed=2018

=> Building model..
=> Loading checkpoint from ./models/state_dict/mobilenetv1_cifar.pth.tar...
=> Preparing data: cifar10...
DEBUG: train/val overlap size: 0
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
=> shape of embedding (n_layer * n_dim): (9, 10)
=> original acc: 100.000%
=> original weight size: 0.8118 M param
=> original FLOPs: 15.1685 M
=> Original model channels: [512, 512, 512, 256, 256, 128, 128, 64, 32]
=> Pruning with ratios: [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.9995041403160194, 0.95]
=> Channels after pruning: [486, 486, 486, 243, 243, 122, 122, 64, 30]
 New 

In [ ]:
!python amc_fine_tune.py \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --batch_size=256 \
    --n_worker=2 \
    --n_epoch=2 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/mobilenet_export.pt

=> Preparing data..
=> Preparing data..
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_export.pt...
=> Model Parameter: 0.737 M, FLOPs: 13.605M
=> Building model..
=> Loading checkpoint from ./checkpoints/mobilenet_export.pt...
=> Resuming from checkpoint..
Using SGD...
weight decay  = 4e-05
=> Start training...
Training mobilenet_cifar on cifar10...
=> Saving logs to ./logs/mobilenet_cifar_cifar10_finetune-run4
=> lr: 0.005

Epoch: 0
 [=======================================>]  Step: 270ms | Tot: 43s697ms | Loss: 0.116 | Acc1: 95.936% | Acc5: 99.986% 196/196 
 [=======================================>]  Step: 42ms | Tot: 2s180ms | Loss: 0.540 | Acc1: 86.290% | Acc5: 99.320% 40/40 
Current best acc: 86.29
=> Saving checkpoint to ./logs/mobilenet_cifar_cifar10_finetune-run4/ckpt.pth.tar
=> lr: 0.0025

Epoch: 1
 [=======================================>]  Step: 12ms | Tot: 45s80ms | Loss: 0.105 | Acc1: 96.406% | Acc5: 99.986% 196/196 
 [===========================

# ResNet

## Download Weights

In [ ]:
!python scripts/download_resnet_weights.py --model resnet18

100% 979M/979M [00:37<00:00, 25.9MB/s]
Download complete.
Extracting only state_dicts/resnet18.pt...
Moved resnet18.pt to models/state_dicts/
Removed zip and temp directory.


In [ ]:
!python amc_search.py \
    --job=train \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --preserve_ratio=0.95 \
    --acc_metric=acc1 \
    --lbound=0.9 \
    --rbound=1 \
    --train_episode=4 \
    --reward=acc_flops_reward \
    --data_root=./data \
    --ckpt_path=./models/state_dicts/resnet18.pt \
    --seed=2018

=> Building model..
=> Loading checkpoint from ./models/state_dicts/resnet18.pt...
=> Preparing data: cifar10...
DEBUG: train/val overlap size: 0
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 99.600%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
=> Saving logs to ./logs/resnet_cifar_cifar10_r0.95_search-run8
=> Output path: ./logs/resnet_cifar_cifar10_r0.95_search-run8...
** Actual replay buffer size: 1100
 New best reward: -0.5022, acc: 97.3000, compress: 0.8485
 New best policy: [0.9, 0.9, 0

In [ ]:
!python amc_search.py \
    --job=export \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --data_root=./data \
    --ckpt_path=./models/state_dicts/resnet18.pt \
    --n_calibration_batches=25 \
    --policy_path=./logs/resnet_cifar_cifar10_r0.95_search-run5/best_policy.txt \
    --export_path=./checkpoints/resnet_export.pt \
    --seed=2018

=> Building model..
=> Loading checkpoint from ./models/state_dicts/resnet18.pt...
=> Preparing data: cifar10...
DEBUG: train/val overlap size: 0
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
=> shape of embedding (n_layer * n_dim): (11, 10)
=> original acc: 100.000%
=> original weight size: 11.1740 M param
=> original FLOPs: 140.8481 M
=> Original model channels: [512, 256, 128, 64, 64, 128, 128, 256, 256, 512, 512]
=> Pruning with ratios: [0.9, 0.9, 0.9, 0.9995041403160194, 0.9, 0.9, 0.9572498787064212, 0.9, 0.9, 0.9, 0.9758796118925429]
=> Channels after pruning: [461, 230, 115, 6

In [ ]:
!python amc_fine_tune.py \
    --model=mobilenet_cifar \
    --dataset=cifar10 \
    --batch_size=256 \
    --n_worker=2 \
    --n_epoch=2 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/mobilenet_export.pt

In [ ]:
!python amc_fine_tune.py \
    --model=resnet_cifar \
    --dataset=cifar10 \
    --batch_size=256 \
    --n_worker=2 \
    --n_epoch=2 \
    --lr=0.005 \
    --lr_type=cos \
    --wd=4e-5 \
    --seed=2025 \
    --data_root=./data \
    --ckpt_path=./checkpoints/resnet_export.pt